In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics

%matplotlib inline

In [14]:
a_row = pd.Series([1, 1])
df = pd.DataFrame([[2, 2], [3, 3]])
df

,0,1
0,2,2
1,3,3


In [15]:
row_df = pd.DataFrame([a_row])
df = pd.concat([df, row_df], ignore_index=True)
df

,0,1
0,2,2
1,3,3
2,1,1
